# Project: Identify Customer Segments

In this project, you will apply unsupervised learning techniques to identify segments of the population that form the core customer base for a mail-order sales company in Germany. These segments can then be used to direct marketing campaigns towards audiences that will have the highest expected rate of returns. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

This notebook will help you complete this task by providing a framework within which you will perform your analysis steps. In each step of the project, you will see some text describing the subtask that you will perform, followed by one or more code cells for you to complete your work. **Feel free to add additional code and markdown cells as you go along so that you can explore everything in precise chunks.** The code cells provided in the base template will outline only the major tasks, and will usually not be enough to cover all of the minor tasks that comprise it.

It should be noted that while there will be precise guidelines on how you should handle certain tasks in the project, there will also be places where an exact specification is not provided. **There will be times in the project where you will need to make and justify your own decisions on how to treat the data.** These are places where there may not be only one way to handle the data. In real-life tasks, there may be many valid ways to approach an analysis task. One of the most important things you can do is clearly document your approach so that other scientists can understand the decisions you've made.

At the end of most sections, there will be a Markdown cell labeled **Discussion**. In these cells, you will report your findings for the completed section, as well as document the decisions that you made in your approach to each subtask. **Your project will be evaluated not just on the code used to complete the tasks outlined, but also your communication about your observations and conclusions at each stage.**

In [ ]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly_express as px
import pandas_profiling as pdp

# magic word for producing visualizations in notebook
%matplotlib inline

# Setup some nice defaults for dataframe display
pd.set_option("display.max_columns", 85)
pd.set_option("display.max_rows", 85)

# Given the expected size of our DataFrames, need to disable jedi for speeding up autocompletion
%config Completer.use_jedi = False

### Step 0: Load the Data

There are four files associated with this project (not including this one):

- `Udacity_AZDIAS_Subset.csv`: Demographics data for the general population of Germany; 891211 persons (rows) x 85 features (columns).
- `Udacity_CUSTOMERS_Subset.csv`: Demographics data for customers of a mail-order company; 191652 persons (rows) x 85 features (columns).
- `Data_Dictionary.md`: Detailed information file about the features in the provided datasets.
- `AZDIAS_Feature_Summary.csv`: Summary of feature attributes for demographics data; 85 features (rows) x 4 columns

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. You will use this information to cluster the general population into groups with similar demographic properties. Then, you will see how the people in the customers dataset fit into those created clusters. The hope here is that certain clusters are over-represented in the customers data, as compared to the general population; those over-represented clusters will be assumed to be part of the core userbase. This information can then be used for further applications, such as targeting for a marketing campaign.

To start off with, load in the demographics data for the general population into a pandas DataFrame, and do the same for the feature attributes summary. Note for all of the `.csv` data files in this project: they're semicolon (`;`) delimited, so you'll need an additional argument in your [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call to read in the data properly. Also, considering the size of the main dataset, it may take some time for it to load completely.

Once the dataset is loaded, it's recommended that you take a little bit of time just browsing the general structure of the dataset and feature summary file. You'll be getting deep into the innards of the cleaning in the first major step of the project, so gaining some general familiarity can help you get your bearings.

In [ ]:
# Load in the general demographics data.
gen_pop = pd.read_csv('data/Udacity_AZDIAS_Subset.csv', sep=';')
gen_pop.head()

In [ ]:
# Load in the feature summary file.
feat_info = pd.read_csv('data/AZDIAS_Feature_Summary.csv', sep=';')
feat_info

In [ ]:
# Check the structure of the data after it's loaded (e.g. print the number of
# rows and columns, print the first few rows).

gen_pop.info(memory_usage='deep')

**That's weird. This says that I have 891,221 rows but the data dictionary and Udacity project page indicate that there should be 10 fewer entries than that, 891,211.** A typo in the data dictionary page perhaps? Or maybe the data have been updated? 

Or perhaps something more nefarious?

In [ ]:
# Do some quick automated EDA
# This will help especially in indicating the original amount of naturally missing data
# prior to mapping the missing/unknown value codes to np.nan

pdp.ProfileReport(gen_pop)


## Step 1: Preprocessing

### Step 1.1: Assess Missing Data

The feature summary file contains a summary of properties for each demographics data column. You will use this file to help you make cleaning decisions during this stage of the project. First of all, you should assess the demographics data in terms of missing data. Pay attention to the following points as you perform your analysis, and take notes on what you observe. Make sure that you fill in the **Discussion** cell with your findings and decisions at the end of each step that has one!

#### Step 1.1.1: Convert Missing Value Codes to NaNs
The fourth column of the feature attributes summary (loaded in above as `feat_info`) documents the codes from the data dictionary that indicate missing or unknown data. While the file encodes this as a list (e.g. `[-1,0]`), this will get read in as a string object. You'll need to do a little bit of parsing to make use of it to identify and clean the data. Convert data that matches a 'missing' or 'unknown' value code into a numpy NaN value. You might want to see how much data takes on a 'missing' or 'unknown' code, and how much data is naturally missing, as a point of interest.

**As one more reminder, you are encouraged to add additional cells to break up your analysis into manageable chunks.**

In [ ]:
# Look at naturally missing data first

missing = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})
missing['percent missing'] = round(missing['total missing'] / len(gen_pop),2)
missing.sort_values('total missing', ascending = False, inplace=True)
missing

In [ ]:
none_missing_count = len(missing[missing['total missing'] > 0])
print(
    f"{none_missing_count} columns have a nonzero number of naturally missing values")


**Oof, looks like the consumer behavior type (`KK_KUNDENTYP`) is pretty sorely lacking values.** Hard to say if it will be the dominant "missing value" feature once we fill in the coded missing values with `np.nan`, but given that it's more than 60% missing I'll probably need to drop it as a feature. In my experience, features with that many missing values are rarely useful for modeling.

In [ ]:
# Convert the strings for the missing values from the feature summary
# To be proper lists of values to use for filling in NaNs

# First, remove brackets
# Then, split on comma separator
feat_info.loc[:, 'missing_or_unknown'] = \
    feat_info.loc[:, 'missing_or_unknown'].str[1:-1].str.split(',')

feat_info


In [ ]:
# What columns are object (probably mixed) type?
gen_pop.columns[gen_pop.dtypes == 'object']

In [ ]:
# What unique data types exist within the object-type columns?
col_types = {}

for col in gen_pop.columns[gen_pop.dtypes == 'object']:
    types = set()
    col_types[col] = set([type(e) for e in gen_pop[col]])
    
col_types

**Good to know! Looks like all object-type columns are of mixed float and str type**. This will help us do intelligent typecasting as we parse through the `feat_info` missing codes below.

A few other notes (ignoring codes used for missing):

1. `OST_WEST_KZ`
    1. Can be "O" or "W"
2. `CAMEO_DEUG_2015`
    1. All int
3. `CAMEO_DEU_2015`
    1. Alphanumeric (of form INT-LETTER)
4. `CAMEO_INTL_2015`
    1. All int

In [ ]:
def fill_missing(df, missing_codes_mapping, inplace=False):
    '''
    Parses dataframe of missing values and their mapping to individual feature names
    and then fills any of those values found in a dataframe's matching feature columns
    with np.nan.

    Inputs
    ------
    df: pandas DataFrame. Table with features that match the ones for which we have
        missing mappings. Each sample is a person.

    missing_codes_mapping: pandas DataFrame. Contains columns 'attribute' and 
        'missing_or_unknown' that map codes used for missing/unknown values to 
        features/attributes. 'missing_or_unknown' is expected to have elements
        that are lists of str (usually ints, but sometimes chars or empty lists).

    Returns
    -------
    df with NaN values filled in according to missing_codes_mapping
    '''

    # Use deep copy if inplace = False, otherwise use shallow copy
    data = df.copy(deep=not inplace)
    missing_codes = missing_codes_mapping.copy(deep=not inplace)

    def parse_missing_codes(code_list):
        '''
        Goes through a list of str and converts the elements of the list according to the needs 
        of the dtypes in our demographic data such that the results can be used for 
        filling in NaN values.

        Inputs
        ------
        code_list: list of str. List is expected to contain the chars, floats, or ints 
            that are codes indicating a missing or unknown value.

        Returns
        -------
        list or np.nan. Each element of the list returned is typecast according to 
            the expected needs of the NaN-filling it will be doing. Empty lists
            (or lists with only an empty string in them) are returned as np.nan.
        '''

        # Make sure list isn't just empty string
        if '' not in code_list:
            # Check if list can be converted to int without issues - if so, do it
            try:
                return [int(e) for e in code_list]

            # Not all are cast-able to int
            except ValueError:
                return [float(e) if 'X' not in e else e for e in code_list]

        else:
            return np.nan

    # Typecast missing value codes appropriately
    missing_codes.loc[:, 'missing_or_unknown'] = \
        missing_codes.loc[:, 'missing_or_unknown'].apply(parse_missing_codes)

    # Create series that maps feature names (index) to missing codes (data)
    code_map = pd.Series(data=missing_codes['missing_or_unknown'].values,
                         index=missing_codes['attribute'].values)

    # When passing a Series into to_replace, index is key and data is value (like a dict)
    data.replace(to_replace=code_map,
                 value=np.nan,
                 inplace=True)

    return data


In [ ]:
# Replace codes we know to mean "missing" or "unknown" with NaN
gen_pop = fill_missing(gen_pop, feat_info)
gen_pop.head()

In [ ]:
# TODO: make sure when importing customer data to use same dtypes as gen_pop

#### Step 1.1.2: Assess Missing Data in Each Column

How much missing data is present in each column? There are a few columns that are outliers in terms of the proportion of values that are missing. You will want to use matplotlib's [`hist()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) function to visualize the distribution of missing value counts to find these columns. Identify and document these columns. While some of these columns might have justifications for keeping or re-encoding the data, for this project you should just remove them from the dataframe. (Feel free to make remarks about these outlier columns in the discussion, however!)

For the remaining features, are there any patterns in which columns have, or share, missing data?

In [ ]:
# Automated EDA again now that we've encoded our missing values as np.nan properly

pdp.ProfileReport(gen_pop)

In [ ]:
# What do the missing data counts look like now, by column?

missing_filled = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})
missing_filled['percent missing'] = round(missing_filled['total missing'] / len(gen_pop),2)
missing_filled.sort_values('total missing', ascending = False, inplace=True)
missing_filled

**Well that's a non-trivial change!** The first profile I generated prior to filling in missing values had 6.4% of all values missing, now we're at 10.6%! Additionally:

1. Before NaN-filling, we only had one feature with more than 15% of values missing (`KK_KUNDENTYP`) and 32 features with no missing values at all. 
    1. After filling in the NaN codes with `np.nan`, we end up with 9 having more than 15% of values missing and 25 with no missing values.
1. `AGER_TYPE` has **77% of its values missing.** It wasn't even flagged as special prior to the NaN-filling step.
2. `ALTER_HH` went from 8.2% missing to **34.8% missing!**
3. Other examples exist (6 more I haven't mentioned that are above 15% missing)

In [ ]:
none_missing_count = len(missing[missing['total missing'] > 0])
print(
    f"{none_missing_count} columns have a nonzero number of missing values now")


In [ ]:
# Visualize the missing count across columns

fig, (ax1, ax2) = plt.subplots(nrows=2)

# Total counts of missing
sns.distplot(gen_pop.isnull().sum(axis=0), bins=50, ax=ax1)
ax1.set(title=f'Distribution of Missing Values in Each Column')

# Percentage missing
sns.distplot(gen_pop.isnull().sum(axis=0)/len(gen_pop), bins=50, ax=ax2)
ax2.set(title=f'Distribution of Missing Values in Each Column', ymargin=-0.4)

plt.tight_layout()

It seems pretty clear here that any features that have more than 200k (20%) missing values are "outliers" in the sense that they are far from the majority of features which are below this threshold. But, that being said, perhaps we should look at each feature first and determine if it might have a lot of relevant information useful for analysis, even if it is substantially lacking values?

In [ ]:
missing_filled[missing_filled['total missing'] > 2E5]

Here are the meanings of these six features:

1. `TITEL_KZ`: Academic Title - categorical
    * Since this one is missing all or nearly all of its values, I see no reason this should be retained
2. `AGER_TYP`: Elderly Type - categorical
    * To be honest, I don't quite know what this one even is. And since it would only apply to a small portion of the dataset (those considered "elderly") it seems like it isn't crucial in terms of providing information, especially since we have other features that provide us with age-related information anyhow.
    * Let's drop it.
3. `KK_KUNDENTYP`: Household-level customer type - categorical
    * As this one has a large amount of missing values (66%) and seems somewhat duplicative with `RETOURTYP_BK_S`, I think we can drop it
4. `KBA05_BAUMAX`: RR3: Most Common Bldg Type - mixed
    * We can probably drop this one too, as it's more than 50% missing and is somewhat duplicative in information with the other `KBA05` features
5. `GEBURTSJAHR`: Birth Year - numeric
    * This is where it gets a bit tougher. I tend not to be too worried about dropping features with more than 50% missing values, but this one is only at 44%. As this is the specific birth year (at least as implied by the data dictionary), it is likely a better measure of the age of each individual in the dataset than even the `ALTERSKATEGORIE_GROB` feature, so we'll retain it
    * I freely admit that I originally intended to drop this one, as it was well outside the majority of features in terms of the distribution of missing values by feature shown in the preceding visualization, but the instructions later in this project made me change my mind (since they indicate I shouldn't have dropped any numeric or interval features). Given that, [upon further research](https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4) I found some guidance to retain a feature and impute its missing values when it has less than 50-60% missing, I was willing to be convinced that this feature should be retained.
6. `ALTER_HH`: Head of Household birthdate (bin) - interval
    * This one is a little less helpful than `GEBURTSJAHR` as it only provides the head of household's birth year in 5-year bins. So given that fact and that it's still significantly more missing values than the rest of the features with less than 200,000 missing values, I'm going to drop it.

In [ ]:
# Remove the outlier columns from the dataset

outlier_cols = ['ALTER_HH', 'KBA05_BAUMAX', 'KK_KUNDENTYP',
                'AGER_TYP', 'TITEL_KZ']
gen_pop.drop(columns=outlier_cols, inplace=True)


In [ ]:
# Re-assess after dropping outlier features

missing_noOutliers = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})
missing_noOutliers['percent missing'] = round(missing_noOutliers['total missing'] / len(gen_pop),2)
missing_noOutliers.sort_values('total missing', ascending = False, inplace=True)
missing_noOutliers

In [ ]:
# What does missing count distribution look like now?

sns.distplot(gen_pop.isnull().sum(axis=0)/len(gen_pop),
            bins = 50)

In [ ]:
# First things first: if I'm going to make sense out of these, I need more helpful names

# Read in the names mapping CSV as a dict
new_names = pd.read_csv('col_renaming.csv', header=None, index_col=0,
                        squeeze=True).to_dict()

gen_pop.rename(columns=new_names, inplace=True)


In [ ]:
# Need to stop sorting by amount missing and leave in original sort order
# so I can more easily compare to the data dictionary file

missing_noSort = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})
missing_noSort['percent missing'] = round(missing_noSort['total missing'] / len(gen_pop),2)

missing_noSort[missing_noSort['total missing'] < 50000]
#missing_noSort[missing_noSort['total missing'] > 50000]

In [ ]:
#missing_noSort[missing_noSort['total missing'] < 50000]
missing_noSort[missing_noSort['total missing'] > 50000]

#### Discussion 1.1.2: Assess Missing Data in Each Column

Some notes on patterns I noticed during my exploration of the missing data as a function of column/feature:

1. Before matching codes for unknown/missing values and filling in those values with `np.nan`, we had 6.4% of all values reported as missing. Afterwards, 10.6% of all values were missing.
2. I removed features with more than 50% missing values as they appeared to be outliers in terms of the amount of missing values relative to the overall feature set and for the reasons discussed above.
3. In the data that are remaining, I see clear breakpoints between features with 0% - 2% missing values, those with 8% - 18% missing, and the two features with 35% and 44% missing values.
    * **Those in the group with 0% - 2% missing:**
        * There are 32 features included in this subset/sample
        * Of those 32, all but two (`RR1: Online Affinity` and `HH: Net Income Bins`) are person-level measures (30, in other words). Given that ~50% of all features are person-level, this suggests that person-level features are overrepresented in the subset (if the information levels were distributed the same in this sample as they are in the total population, we'd expect only 16-17 features to be person-level). 
            * Also, this means that 70% of the total 43 possible person-level features in the full feature space are represented in this sample.
        * Given that Head of Household birthdate and individual birth year both had very high missing data counts (causing them to be dropped as outliers), it's surprising to see `Age Bin` with such a relatively low number of missing values. That seems really odd, but likely it is due to the method used for determining the age of an individual. The `Age Bin` values cover 15-30 year chunks of a person's life, whereas the birth year and head of household birth date features had resolutions of 1 and 5 years, resp.
        * It's also interesting to note that `Consumption Channel Type`, `Vacation Habits`, `Customer Type`, `Socioeconomic Status - HighRes`, `Socioeconomic Status - LowRes`, and `Online Affinity` all have the exact same number of missing values. This suggests to me that they all came from the same data source and likely used the same methodology for their derivation, as this pattern seems a little too consistent to be random.
    * **Those in the group with 8% - 18% missing:**
        * There are 47 features included in this sample/subset
        * Of those 47, 10 (21%) are person-level features, suggesting that missing/unknown data are much more common at the higher-level resolutions than at the person-level. 
            * This strikes me as odd, given especially that many of those person-level feature values were likely not directly measured, but rather derived from higher-level datasets (e.g. statistically modeled from census data).
        * It seems particularly odd that so many of the features that have higher missing value rates seem to be highly measurable quantities (e.g. from census/GIS data, such as the number of inhabitants in a community).
    * **I already discussed my thoughts on those with 35% and 44% missing.**
        * I retained the 44% missing birth year feature, as it seems to provide significant information useful for later analysis
        * One other thought: these are age-related variables, one at the person-level and one at the household-level. It seems odd to me that the `Age Bin` feature should be in the lowest missing value count group and these in the highest. Even though they have significantly higher resolutions than the `Age Bin` feature, they must still be drawn from very different data sources, since they shouldn't be *so* different. Strange.

#### Step 1.1.3: Assess Missing Data in Each Row

Now, you'll perform a similar assessment for the rows of the dataset. How much data is missing in each row? As with the columns, you should see some groups of points that have a very different numbers of missing values. Divide the data into two subsets: one for data points that are above some threshold for missing values, and a second subset for points below that threshold.

In order to know what to do with the outlier rows, we should see if the distribution of data values on columns that are not missing data (or are missing very little data) are similar or different between the two groups. Select at least five of these columns and compare the distribution of values.
- You can use seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) function to create a bar chart of code frequencies and matplotlib's [`subplot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplot.html) function to put bar charts for the two subplots side by side.
- To reduce repeated code, you might want to write a function that can perform this comparison, taking as one of its arguments a column to be compared.

Depending on what you observe in your comparison, this will have implications on how you approach your conclusions later in the analysis. If the distributions of non-missing features look similar between the data with many missing values and the data with few or no missing values, then we could argue that simply dropping those points from the analysis won't present a major issue. On the other hand, if the data with many missing values looks very different from the data with few or no missing values, then we should make a note on those data as special. We'll revisit these data later on. **Either way, you should continue your analysis for now using just the subset of the data with few or no missing values.**

In [ ]:
# How many duplicate rows are there?

dups = len(gen_pop) - len(gen_pop.drop_duplicates())

print(f"There are {dups} duplicate rows in the dataset, \
representing {round(dups/len(gen_pop)*100,2)}% of all samples.")

**This is a non-trivial fraction of data that are duplicates.** However, since we don't have person-level IDs for each record, it's impossible to tell if these are truly duplicates or merely extremely similar people. As such, we'll leave these alone.

In [ ]:
# What do the missing data counts look like now, by row?

rows_oneMissing_filled = len(gen_pop) - len(gen_pop.dropna(how='any'))
rows_allMissing_filled = len(gen_pop) - len(gen_pop.dropna(how='all'))

print(f"There are {rows_oneMissing_filled} samples with at least one feature missing a value \
\nand {rows_allMissing_filled} samples missing values for all features")

In [ ]:
# How much data is missing in each row of the dataset?

fig, ax = plt.subplots()
fig.suptitle('Fraction of Different Amounts of Missing Values Across Rows')

sns.distplot(gen_pop.isnull().sum(axis=1), kde=False, norm_hist=True, ax=ax)
ax.set(xlabel='Number of Columns with Missing Values')

In [ ]:
# How much data is missing in each row of the dataset?

fig, ax = plt.subplots()
fig.suptitle('Fraction of Different Amounts of Missing Values Across Rows')

sns.distplot(gen_pop.isnull().sum(axis=1), kde=False, norm_hist=True, ax=ax)
ax.set(xlabel='Number of Columns with Missing Values', xlim=(0,5))

**Well, it looks like a pretty safe bet to say that we should subset the data into rows with up to one missing value and rows with 2+ missing values,** since the former comprises around 70% of the data it seems.

In [ ]:
# Write code to divide the data into two subsets based on the number of missing
# values in each row.

gen_pop_lowMissing = gen_pop.loc[gen_pop.isnull().sum(axis=1) < 2,:]
gen_pop_highMissing = gen_pop.loc[gen_pop.isnull().sum(axis=1) >= 2,:]


# Let's just check that our numbers work out as we expct them to
print(f"There are {len(gen_pop_lowMissing)} rows that have one or fewer missing values")
print(f"There are {len(gen_pop_highMissing)} rows that have 2+ missing values")
print(f"There are {len(gen_pop_lowMissing) + len(gen_pop_highMissing)} \
rows when you combine these two")
print(f"\nThere are {len(gen_pop)} rows in the full dataset")

In [ ]:
# Compare the distribution of values for at least five columns where there are
# no or few missing values, between the two subsets.

def dist_compare(df, missing_count, column):
    '''
    Compares the distribution of values for a single feature 
    between two subsets of a DataFrame, subsetting based upon the number of missing values
    allowed per row (e.g. one distribution is only of rows with less than X missing values,
    the other is only of rows with at least X missing values).
    
    Inputs
    ------
    df: pandas DataFrame in long format
        
    missing_count: int. Number of missing values to use as breakpoint for subsetting df
    
    column: str. Column name of feature from the DataFrames that you're investigating.
    
    
    Returns
    -------
    Matplotlib figure comparing the two distributions.
    '''

    fig, (ax_lowMissing, ax_highMissing) = plt.subplots(ncols=2)
    fig.suptitle(f'Distributions of Mostly Complete Features - {column}\
    \nMissing Values Breakpoint = {missing_count}', 
                 fontsize=15, y=1.2)
    plt.figure(figsize=(20,5))

    
    sns.distplot(df.loc[df.isnull().sum(axis=1) < missing_count, column].dropna(), 
                 ax=ax_lowMissing)
    ax_lowMissing.set(title=f'Rows with Fewer \nMissing Values')

    sns.distplot(df.loc[df.isnull().sum(axis=1) >= missing_count, column].dropna(), 
                 ax=ax_highMissing)
    ax_highMissing.set(title=f'Rows with More \nMissing Values')

In [ ]:
# The features that are completely or almost completely filled
# and likely to show more than just two values in the distribution

full_cols = ['Energy Consumption Type', 'MoneyType__Primary',
            'Age Bin', 'Customer Type', 'RR1: Online Affinity',
            'Vacation Habits']

In [ ]:
missing_values_breakpoint = 2

In [ ]:
dist_compare(gen_pop, missing_values_breakpoint, 'Energy Consumption Type')

It's clear from these plots of `Energy Consumption Type` that, while both subsets seem to have a majority of people with an energy consumption pattern of 3 ("fair supplied"), this majority is a larger fraction of the total subset for the data with a higher missing value count.

In [ ]:
dist_compare(gen_pop, missing_values_breakpoint, 'MoneyType__Primary')

Much like with `Energy Consumption Type`, there seems to be a dominant feature value for `MoneyType__Primary` when you look at the subset with a high missing value count. In this case, it's code 4: "be prepared". There's also a clearly dominant code for the low missing values data: code 6 ("inconspicuous").

In [ ]:
dist_compare(gen_pop, missing_values_breakpoint, 'Age Bin')

**This one is a little more subtle than the preceding two.** For the most part, the two subset distributions for `Age Bin` look the same, with the only notable difference being that code 4 (> 60 years old) is a smaller fraction of the overall distribution for the high missing values subset than it is for the low missing values one.

In [ ]:
dist_compare(gen_pop, missing_values_breakpoint, 'Customer Type')

As with `Age Bin`, `Customer Type` has a more subtle distributional difference between the two ssubsets. That said, it looks like it would be fair to say that code 5 ("determined Minimal-Returner") is the dominant feature for the low missing values subset and code 3 ("incentive-receptive Normal-Returner")

In [ ]:
dist_compare(gen_pop, missing_values_breakpoint, 'RR1: Online Affinity')

For `RR1: Online Affinity`, there can be no doubt: code 2 ("middle") is the dominant feature for the high missing values subset, and the codes are pretty uniformly distributed for the low missing values subset.

In [ ]:
dist_compare(gen_pop, missing_values_breakpoint, 'Vacation Habits')

For `Vacation Habits`, the pattern is clear: code 5 (nature fans) is the dominant feature for the high missing values subset and the distribution is pretty uniform across codes (although with a dip for codes 6 and 7 and an interesting overall positive slope) for the low missing values subset.

#### Discussion 1.1.3: Assess Missing Data in Each Row

**Ultimately, it seems like the distributions are more uniform for the rows with low missing value counts than they are for the rows with high missing value counts.** As such, it's important to note that the distribution, at least of these features we spot-checked, is not so similar between the two subsets that we can easily say we've lost no information by excluding the rows with higher amounts of missing values. 

### Step 1.2: Select and Re-Encode Features

Checking for missing data isn't the only way in which you can prepare a dataset for analysis. Since the unsupervised learning techniques to be used will only work on data that is encoded numerically, you need to make a few encoding changes or additional assumptions to be able to make progress. In addition, while almost all of the values in the dataset are encoded using numbers, not all of them represent numeric values. Check the third column of the feature summary (`feat_info`) for a summary of types of measurement.
- For numeric and interval data, these features can be kept without changes.
- Most of the variables in the dataset are ordinal in nature. While ordinal values may technically be non-linear in spacing, make the simplifying assumption that the ordinal variables can be treated as being interval in nature (that is, kept without any changes).
- Special handling may be necessary for the remaining two variable types: categorical, and 'mixed'.

In the first two parts of this sub-step, you will perform an investigation of the categorical and mixed-type features and make a decision on each of them, whether you will keep, drop, or re-encode each. Then, in the last part, you will create a new data frame with only the selected and engineered columns.

Data wrangling is often the trickiest part of the data analysis process, and there's a lot of it to be done here. But stick with it: once you're done with this step, you'll be ready to get to the machine learning parts of the project!

In [ ]:
# Add new attribute names to feat_info

feat_info['attribute_renamed'] = pd.read_csv('col_renaming.csv', header=None)[1]
feat_info

In [ ]:
# How many features are there of each data type?

# I'm only looking at the features that still remain in gen_pop after dropping outlier features
feat_info_remaining = feat_info.loc[feat_info['attribute_renamed'].isin(gen_pop.columns), :]
feat_info_remaining['type'].value_counts()

#### Step 1.2.1: Re-Encode Categorical Features

For categorical data, you would ordinarily need to encode the levels as dummy variables. Depending on the number of categories, perform one of the following:
- For binary (two-level) categoricals that take numeric values, you can keep them without needing to do anything.
- There is one binary variable that takes on non-numeric values. For this one, you need to re-encode the values as numbers or create a dummy variable.
- For multi-level categoricals (three or more values), you can choose to encode the values using multiple dummy variables (e.g. via [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)), or (to keep things straightforward) just drop them from the analysis. As always, document your choices in the Discussion section.

In [ ]:
# Assess categorical variables: which are binary, which are multi-level, and
# which one needs to be re-encoded?

feat_info_remaining[feat_info_remaining['type'] == 'categorical']

In [ ]:
# Figure out what different values are available for each feature

cols = feat_info_remaining.loc[feat_info_remaining['type'] == 'categorical', 'attribute_renamed']

unique_cat_vals = pd.DataFrame()

for col in cols:
    temp = pd.DataFrame({'attribute': col, 'unique values': gen_pop[col].unique()})
    unique_cat_vals = pd.concat([unique_cat_vals, temp])
    
unique_cat_vals.groupby(['attribute']).count().sort_values('unique values')

In [ ]:
# What are the values taken on by the binary features?

binary_cols = unique_cat_vals.groupby(['attribute']).count().sort_values('unique values').index[:5]
unique_cat_vals[unique_cat_vals['attribute'].isin(binary_cols)]

**The following features appear to be binary in nature:**

1. `Gender`
1. `Young Environmentalist`
2. `Bldg: Location Relative to E or W Germany`
    * This one takes on values of 'W' and 'O' for West and East Germany, resp.
    * Given it's non-numeric nature, we need to use dummy variables with this
3. `Small or Home Office Owner`
    * This one actually has `NaN` values, which means I need to use dummy variables with it too (so `NaN` can be reflected by `0,0`)
5. `Insurance Type`
    * This one also has `NaN` values, so I'll need to use a dummy with it too!

**That leaves the following features for re-encoding through dummies:**

1. `Nationality Based on Name`
2. `Shopper Type`
3. `Socioeconomic Status - LowRes`
4. `Family Type - LowRes`
5. `MoneyType__Primary`
6. `Energy Consumption Type`
7. `Consumption Channel Type`
8. `Bldg: Building Type`
9. `RR4: Life Stage Type - LowRes`
10. `Socioeconomic Status - HighRes`
11. `Family Type - HighRes`
12. `Vacation Habits`
13. `RR4: Life Stage Type - HighRes`

In [ ]:
# Re-encode categorical variable(s) to be kept in the analysis.

cat_cols = ['Bldg: Location Relative to E or W Germany',
           'Small or Home Office Owner', 'Insurance Type',
           'Nationality Based on Name', 'Shopper Type',
            'Socioeconomic Status - LowRes', 'Family Type - LowRes',
            'MoneyType__Primary', 'Energy Consumption Type',
            'Consumption Channel Type', 'Bldg: Building Type',
            'RR4: Life Stage Type - LowRes', 'Socioeconomic Status - HighRes',
            'Family Type - HighRes', 'Vacation Habits', 
            'RR4: Life Stage Type - HighRes']

# Have to drop first dummy so avoid Dummy Variable Trap
# Have to include NaN dummy so zero-vector can't be interpreted ambiguously
    # as either NaN or first dropped dummy (which are likely different)
gen_pop_lowMissing = pd.get_dummies(
    gen_pop_lowMissing, prefix_sep='__', 
    drop_first=True, dummy_na=True,
    columns=cat_cols)

gen_pop_lowMissing.columns

In [ ]:
# What does our dataframe look like now?

gen_pop_lowMissing.info()

In [ ]:
gen_pop_lowMissing.dtypes.value_counts()

In [ ]:
gen_pop_lowMissing.columns[gen_pop_lowMissing.dtypes == 'object']

#### Discussion 1.2.1: Re-Encode Categorical Features

**I treated these features as binary and not requiring further engineering:**

1. `Gender`
1. `Young Environmentalist`

**I treated these features as binary but requiring dummy variables due to their encoding or due to the presence of `NaN` values:**

1. `Bldg: Location Relative to E or W Germany`
3. `Small or Home Office Owner`
5. `Insurance Type`

**And I used dummies to encode these multi-level categoricals:**

1. `Nationality Based on Name`
2. `Shopper Type`
3. `Socioeconomic Status - LowRes`
4. `Family Type - LowRes`
5. `MoneyType__Primary`
6. `Energy Consumption Type`
7. `Consumption Channel Type`
8. `Bldg: Building Type`
9. `RR4: Life Stage Type - LowRes`
10. `Socioeconomic Status - HighRes`
11. `Family Type - HighRes`
12. `Vacation Habits`
13. `RR4: Life Stage Type - HighRes`

#### Step 1.2.2: Engineer Mixed-Type Features

There are a handful of features that are marked as "mixed" in the feature summary that require special treatment in order to be included in the analysis. There are two in particular that deserve attention; the handling of the rest are up to your own choices:
- "PRAEGENDE_JUGENDJAHRE" combines information on three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west). While there aren't enough levels to disentangle east from west, you should create two new variables to capture the other two dimensions: an interval-type variable for decade, and a binary variable for movement.
- "CAMEO_INTL_2015" combines information on two axes: wealth and life stage. Break up the two-digit codes by their 'tens'-place and 'ones'-place digits into two new ordinal variables (which, for the purposes of this project, is equivalent to just treating them as their raw numeric values).
- If you decide to keep or engineer new features around the other mixed-type features, make sure you note your steps in the Discussion section.

Be sure to check `Data_Dictionary.md` for the details needed to finish these tasks.

**Mapping for `Generation Designation` Re-Encoding:**

Original Code | Original Code Meaning | Decade Code | Movement Code
--- | --- | --- | ---
1 | 40s - war years (Mainstream, E+W) | 0 | 0
2 | 40s - reconstruction years (Avantgarde, E+W) | 0 | 1
3 | 50s - economic miracle (Mainstream, E+W) | 1 | 0
4 | 50s - milk bar / Individualisation (Avantgarde, E+W) | 1 | 1
5 | 60s - economic miracle (Mainstream, E+W) | 2 | 0
6 | 60s - generation 68 / student protestors (Avantgarde, W) | 2 | 1
7 | 60s - opponents to the building of the Wall (Avantgarde, E) | 2 | 1
8 | 70s - family orientation (Mainstream, E+W) | 3 | 0
9 | 70s - peace movement (Avantgarde, E+W) | 3 | 1
10 | 80s - Generation Golf (Mainstream, W) | 4 | 0
11 | 80s - ecological awareness (Avantgarde, W) | 4 | 1
12 | 80s - FDJ / communist party youth organisation (Mainstream, E) | 4 | 0
13 | 80s - Swords into ploughshares (Avantgarde, E) | 4 | 1
14 | 90s - digital media kids (Mainstream, E+W) | 5 | 0
15 | 90s - ecological awareness (Avantgarde, E+W) | 5 | 1

In [ ]:
def data_mapper(series, mapping_dict):
    '''
    Reads in a pandas Series object that represents the Generation Designation feature
    and returns a re-encoded series according to mapping_dict.

    Inputs
    ------
    series: pandas Series of integer codes (1 through 15) representing different
        Generation Designation values

    mapping_dict: dict of form {designation_code: new_code} used to determine what
        values to return


    Returns
    -------
    pandas Series with the new codes
    '''
    
    # Since NaN values aren't always propagated as expected, do a quick check
    print(f"There are {series.isnull().sum()} null values in the series \
    {series.name} prior to extraction")
    
    out = series.map(mapping_dict, na_action = 'ignore')
    
    print(f"There are {out.isnull().sum()} null values in the series \
    {series.name} after extraction")
    
    return out              


In [ ]:
decade_code_map = {
    1: 0,
    2: 0,
    3: 1,
    4: 1,
    5: 2,
    6: 2,
    7: 2,
    8: 3,
    9: 3,
    10: 4,
    11: 4,
    12: 4,
    13: 4,
    14: 5,
    15: 5
}

In [ ]:
# Investigate "PRAEGENDE_JUGENDJAHRE" and engineer two new variables.

# Variable 1: generation by decade
gen_pop_lowMissing['Generation Decade'] = \
    data_mapper(gen_pop_lowMissing['Generation Designation'],
                              decade_code_map)


In [ ]:
# Does it seem like this worked?
# Should be min of 0 and max of 5 

gen_pop_lowMissing['Generation Decade'].describe()

In [ ]:
gen_pop_lowMissing['Generation Decade'].name

In [ ]:
movement_code_map = {
    1: 0,
    2: 1,
    3: 0,
    4: 1,
    5: 0,
    6: 1,
    7: 1,
    8: 0,
    9: 1,
    10: 0,
    11: 1,
    12: 0,
    13: 1,
    14: 0,
    15: 1
}


In [ ]:
# Feature: "PRAEGENDE_JUGENDJAHRE"

# Variable 2: movement type
gen_pop_lowMissing['Generation Movement'] = \
    data_mapper(gen_pop_lowMissing['Generation Designation'],
                              movement_code_map)

In [ ]:
# Does it seem like this worked?
# Should be min of 0 and max of 1

gen_pop_lowMissing['Generation Movement'].describe()

In [ ]:
# Drop Generation Designation feature now that we've captured its information in the new features
gen_pop_lowMissing.drop(columns='Generation Designation', inplace=True)

**Mapping for `RR4: Life Stage Type - Int'l Code Mapping` Re-Encoding:**

Original Code | Original Code Meaning | Wealth Code | Life Stage Code
--- | --- | --- | ---
11 | Wealthy Households - Pre-Family Couples & Singles | 1 | 1
12 | Wealthy Households - Young Couples With Children | 1 | 2
13 | Wealthy Households - Families With School Age Children | 1 | 3
14 | Wealthy Households - Older Families & Mature Couples | 1 | 4
15 | Wealthy Households - Elders In Retirement | 1 | 5
21 | Prosperous Households - Pre-Family Couples & Singles | 2 | 1
22 | Prosperous Households - Young Couples With Children | 2 | 2
23 | Prosperous Households - Families With School Age Children | 2 | 3
24 | Prosperous Households - Older Families & Mature Couples | 2 | 4
25 | Prosperous Households - Elders In Retirement | 2 | 5
31 | Comfortable Households - Pre-Family Couples & Singles | 3 | 1
32 | Comfortable Households - Young Couples With Children | 3 | 2
33 | Comfortable Households - Families With School Age Children | 3 | 3
34 | Comfortable Households - Older Families & Mature Couples | 3 | 4
35 | Comfortable Households - Elders In Retirement | 3 | 5
41 | Less Affluent Households - Pre-Family Couples & Singles | 4 | 1
42 | Less Affluent Households - Young Couples With Children | 4 | 2
43 | Less Affluent Households - Families With School Age Children | 4 | 3
44 | Less Affluent Households - Older Families & Mature Couples | 4 | 4
45 | Less Affluent Households - Elders In Retirement | 4 | 5
51 | Poorer Households - Pre-Family Couples & Singles | 5 | 1
52 | Poorer Households - Young Couples With Children | 5 | 2
53 | Poorer Households - Families With School Age Children | 5 | 3
54 | Poorer Households - Older Families & Mature Couples | 5 | 4
55 | Poorer Households - Elders In Retirement | 5 | 5

In [ ]:
# Investigate "CAMEO_INTL_2015" and engineer two new variables.

# "CAMEO_INTL_2015" combines information on two axes: wealth and life stage. 
# Break up the two-digit codes by their 'tens'-place and 'ones'-place digits into two new 
# ordinal variables (which, for the purposes of this project, is equivalent to just treating 
# them as their raw numeric values).

# Cast as int since it seems to keep coming up as object type
gen_pop_lowMissing["RR4: Life Stage Type - Int'l Code Mapping"] =\
gen_pop_lowMissing["RR4: Life Stage Type - Int'l Code Mapping"].astype(int)

# Extract the tens digit - wealth code
gen_pop_lowMissing["RR4: Life Stage Type - Int'l - Wealth"] = \
(gen_pop_lowMissing["RR4: Life Stage Type - Int'l Code Mapping"] / 10).astype(int)

# Extract the ones digit - life stage code
gen_pop_lowMissing["RR4: Life Stage Type - Int'l - Stage"] = \
gen_pop_lowMissing["RR4: Life Stage Type - Int'l Code Mapping"] % 10

In [ ]:
# Should be between 1 and 5
gen_pop_lowMissing["RR4: Life Stage Type - Int'l - Wealth"].describe()

In [ ]:
# Should be between 1 and 5
gen_pop_lowMissing["RR4: Life Stage Type - Int'l - Stage"].describe()

In [ ]:
# Drop lngering RR4: Life Stage Type - Int'l Code Mapping feature
gen_pop_lowMissing.drop(columns="RR4: Life Stage Type - Int'l Code Mapping", inplace=True)

In [ ]:
# What other features do I need to look at?

# Ignoring the two features I just worked on
feat_info_remaining[feat_info_remaining['type'] == 'mixed'].drop(labels=[22,59])

**As you're typically better off when modeling to have more features instead of fewer ones (at least prior to feature selection and dimensionality reduction), especially if there is somewhat duplicative data within an original feature, I'm going to opt to split these features up whenever possible.**

Here are my thoughts

1. `Life Stage - HighRes` and ` - LowRes`: lots of info, but inconsistent levels
    * These have information related to age, marital status, income level, homeownership status, and (birth?) household type, making them potentially rich for new generating new features
    * *However*, the levels are very inconsistent across the codes, making it impossible to pull out the same information across all original codes 
        * Even if we were willing to drop information like we did earlier for the `Generation Designation` feature (East vs. West, if you recall), we couldn't do that here since some values have a completely different set of levels than others do.
    * As a result of all of this, I'm going to be treating these as though they're categorical, making dummies out of all of them
2. `Bldg: Neighborhood Quality`: non-obvious ordering of values
    * `Bldg: Neighborhood Quality` seems at first glance to be clearly ordinal, until you realize that "rural" and "new building rural" aren't inherently *less than* a "very poor neighborhood"
    * The plan: we'll split this into a Rural categorical (dummy) feature and set any observations with the rural codes in the original feature to np.nan, so we can maintain the ordinal information as much as possible as well as the rural/urban split information, even though we won't have any quality information on the rural buildings (which we never had to begin with!)
3. `PLZ8: Most Common Bldg Type`: mixed ordering
    * If I'm interpreting this one correctly, it refers to the size of buildings (e.g. how many families can be housed within one). That would seem to me to be an indication of square footage, but since multi-family buildings are measured differently than businesses (e.g. a convenience store is much smaller than a 10-family home, but a grocery store is much larger), it's not appropriate to keep them on the same ordinal scale.
    * That being said, the other `PLZ8` features essentially capture the scale of majority building type if it is a certain size of multi-family home. The only new information provided in this feature is if it is primarily business buildings, so we'll simply make it a binary feature indicating if business buildings are the primary property type or not.

In [ ]:
# First, the easy ones: dummies for Life Stage - HighRes, and Life Stage - LowRes

cat_cols = ['Life Stage - HighRes',
           'Life Stage - LowRes']

gen_pop_lowMissing = pd.get_dummies(
    gen_pop_lowMissing, prefix_sep='__', 
    drop_first=True, dummy_na=True,
    columns=cat_cols)

gen_pop_lowMissing.info()

**Mapping for `Bldg: Neighborhood Quality` Re-Encoding:**

Original Code | Original Code Meaning | Neighborhood Quality Code | Rural Code
--- | --- | --- | ---
0 | no score calculated | 0 | np.nan
1 | very good neighborhood | 1 | 0
2 | good neighborhood | 2 | 0
3 | average neighborhood | 3 | 0
4 | poor neighborhood | 4 | 0
5 | very poor neighborhood | 5 | 0
7 | rural neighborhood | 0 | 1
8 | new building in rural neighborhood | 0 | 2

In [ ]:
# Map the rural codes to a new feature

rural_code_map = {
    0: np.nan,
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    7: 1,
    8: 2
}

gen_pop_lowMissing['Bldg: Rural Type'] = \
    data_mapper(gen_pop_lowMissing['Bldg: Neighborhood Quality'],
                rural_code_map)


In [ ]:
# Make dummies out of the multi-level Rural Type categorical
cat_cols = ['Bldg: Rural Type']

gen_pop_lowMissing = pd.get_dummies(gen_pop_lowMissing,
                                    prefix_sep='__',
                                    drop_first=True,
                                    dummy_na=True,
                                    columns=cat_cols)


In [ ]:
# Make the rural codes effectively unscored
neighborhood_code_map = {
    0: 0,
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    7: 0,
    8: 0
}

gen_pop_lowMissing['Bldg: Neighborhood Quality'] = \
    data_mapper(gen_pop_lowMissing['Bldg: Neighborhood Quality'],
                              neighborhood_code_map)

**Mapping for `PLZ8: Most Common Bldg Type` Re-Encoding:**

Original Code | Original Code Meaning | Business Bldg Code
--- | --- | ---
1 | mainly 1-2 family homes | 0
2 | mainly 3-5 family homes | 0
3 | mainly 6-10 family homes | 0
4 | mainly 10+ family homes | 0
5 | mainly business buildings | 1

In [ ]:
biz_code_map = {
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 1
}

gen_pop_lowMissing['PLZ8: Primarily Business Bldgs'] = \
    data_mapper(gen_pop_lowMissing['PLZ8: Most Common Bldg Type'],
                              biz_code_map)


In [ ]:
# Drop original feature, it's no longer useful
gen_pop_lowMissing.drop(columns=['PLZ8: Most Common Bldg Type'],
                         inplace=True)

#### Discussion 1.2.2: Engineer Mixed-Type Features

Summarizing all of my earlier notes on the mixed-type features:

1. `Generation Designation` (`PRAEGENDE_JUGENDJAHRE`) was split into a feature focused on the decade of that person's generation and the movement they were associated with from that decade (avantgarde or mainstream)
2. For `RR4: Life Stage Type - Int'l Code Mapping` (`CAMEO_INTL_2015`), I split the feature into two new features that extracted information from the original feature about a person's wealth level and their current life stage.
3. As they didn't seem to have any meaningful ordinal/interval/numeric data that I needed to salvage, I decided to treat `Life Stage - HighRes` and `Life Stage - LowRes` as categorical variables and made dummies from them
4. With `Bldg: Neighborhood Quality`, I made the assumption based on the levels of this variable that likely any codes that didn't mention rural buildings reflected areas that were not primarily rural. As such, I extracted a new multi-level categorical feature that indicates if a person's building is primarily in a rural neighborhood, in a rural neighborhood but a new building, or not in a rural neighborhood at all. I then made dummies out of this new feature. I otherwise left the feature as it was, since the neighborhood quality score (presumably for non-rural neighborhoods only) is of interval type (or at least ordinal).
5. For our final mixed-type feature, `PLZ8: Most Common Bldg Type`, I investigated the feature's allowed levels a bit and determined that it was mostly redundant with the other PLZ8 features we already have. That being said, I determined that it shouldn't be dropped as it contained information about how prominent business buildings are in the PLZ8 region, something I didn't see reflected in the other features. So I stripped that information out of the feature as a binary feature unto itself and dropped the rest of the information.

#### Step 1.2.3: Complete Feature Selection

In order to finish this step up, you need to make sure that your data frame now only has the columns that you want to keep. To summarize, the dataframe should consist of the following:
- All numeric, interval, and ordinal type columns from the original dataset.
- Binary categorical features (all numerically-encoded).
- Engineered features from other multi-level categorical features and mixed features.

Make sure that for any new columns that you have engineered, that you've excluded the original columns from the final dataset. Otherwise, their values will interfere with the analysis later on the project. For example, you should not keep "PRAEGENDE_JUGENDJAHRE", since its values won't be useful for the algorithm: only the values derived from it in the engineered features you created should be retained. As a reminder, your data should only be from **the subset with few or no missing values**.

In [ ]:
# Do whatever you need to in order to ensure that the dataframe only contains
# the columns that should be passed to the algorithm functions.

gen_pop_lowMissing.columns.values

The primary thing I was worried about was remembering to drop features that we had extracted information out of. I don't want a bunch of redundant information lying about during modeling. Looking through the column names, **it appears I've managed to avoid this, so I'm satisfied!**

That being said, I'm also going to quickly investigate the distribution of missing values by columns and rows again, since I added some NaN values as part of my feature engineering work. 

In [ ]:
# Check the columns

fig, (ax1, ax2) = plt.subplots(nrows=2)

# Total counts of missing
sns.distplot(gen_pop.isnull().sum(axis=0),
             kde=False, norm_hist=False, bins=50, ax=ax1)
ax1.set(title=f'Distribution of Missing Values in Each Column')

# Percentage missing
sns.distplot(gen_pop.isnull().sum(axis=0)/len(gen_pop), 
             kde=False, norm_hist=False, bins=50, ax=ax2)
ax2.set(title=f'Distribution of % Missing Values in Each Column')

plt.tight_layout()

In [ ]:
s = gen_pop.isnull().sum(axis=0)
s[s>0]

**OK, so the only meaningful one is `Birth Year`, which is no different than what we had before.** The only difference is that, at this point in our investigation, we've dropped rows, so now the missing value for `Birth Year` is now 37.5% of all rows, lower than before! 

In [ ]:
# Check the rows

fig, ax = plt.subplots()
fig.suptitle('Fraction of Different Amounts of Missing Values Across Rows')

sns.distplot(gen_pop.isnull().sum(axis=1), kde=False, norm_hist=False, ax=ax)
ax.set(xlabel='Number of Columns with Missing Values')

**Fantastic! We still only have rows with zero or one missing values in them.** That works for me.

### Step 1.3: Create a Cleaning Function

Even though you've finished cleaning up the general population demographics data, it's important to look ahead to the future and realize that you'll need to perform the same cleaning steps on the customer demographics data. In this substep, complete the function below to execute the main feature selection, encoding, and re-engineering steps you performed above. Then, when it comes to looking at the customer data in Step 3, you can just run this function on that DataFrame to get the trimmed dataset in a single step.

In [ ]:
def clean_data(df, feature_summary):
    """
    Perform feature trimming, re-encoding, and engineering for demographics
    data
    
    INPUTS
    ------
    df: Demographics DataFrame
    
    feature_summary: DataFrame that includes listing of the various missing value codes
        for each feature
        
    OUTPUT: Trimmed and cleaned demographics DataFrame
    """
    
    # -------------------------------------------------------------------
    # convert missing value codes into NaNs, ...

    # Convert the strings for the missing values from the feature summary
    # To be proper lists of values to use for filling in NaNs

    # First, remove brackets
    # Then, split on comma separator
    feature_summary.loc[:, 'missing_or_unknown'] = \
        feature_summary.loc[:, 'missing_or_unknown'].str[1:-1].str.split(',')



    def fill_missing(df, missing_codes_mapping, inplace=False):
        '''
        Parses dataframe of missing values and their mapping to individual feature names
        and then fills any of those values found in a dataframe's matching feature columns
        with np.nan.

        Inputs
        ------
        df: pandas DataFrame. Table with features that match the ones for which we have
            missing mappings. Each sample is a person.

        missing_codes_mapping: pandas DataFrame. Contains columns 'attribute' and 
            'missing_or_unknown' that map codes used for missing/unknown values to 
            features/attributes. 'missing_or_unknown' is expected to have elements
            that are lists of str (usually ints, but sometimes chars or empty lists).

        Returns
        -------
        df with NaN values filled in according to missing_codes_mapping
        '''

        # Use deep copy if inplace = False, otherwise use actual inputs
        if inplace:
            data = df
            missing_codes = missing_codes_mapping
        else:            
            data = df.copy(deep=True)
            missing_codes = missing_codes_mapping.copy(deep=True)

        def parse_missing_codes(code_list):
            '''
            Goes through a list of str and converts the elements of the list according to the needs 
            of the dtypes in our demographic data such that the results can be used for 
            filling in NaN values.

            Inputs
            ------
            code_list: list of str. List is expected to contain the chars, floats, or ints 
                that are codes indicating a missing or unknown value.

            Returns
            -------
            list or np.nan. Each element of the list returned is typecast according to 
                the expected needs of the NaN-filling it will be doing. Empty lists
                (or lists with only an empty string in them) are returned as np.nan.
            '''

            # Make sure list isn't just empty string
            if '' not in code_list:
                # Check if list can be converted to int without issues - if so, do it
                try:
                    return [int(e) for e in code_list]

                # Not all are cast-able to int
                except ValueError:
                    return [float(e) if 'X' not in e else e for e in code_list]

            else:
                return np.nan

        # Typecast missing value codes appropriately
        missing_codes.loc[:, 'missing_or_unknown'] = \
            missing_codes.loc[:, 'missing_or_unknown'].apply(parse_missing_codes)

        # Create series that maps feature names (index) to missing codes (data)
        code_map = pd.Series(data=missing_codes['missing_or_unknown'].values,
                             index=missing_codes['attribute'].values)

        # When passing a Series into to_replace, index is key and data is value (like a dict)
        data.replace(to_replace=code_map,
                     value=np.nan,
                     inplace=True)

        return data
    

    df = fill_missing(df, feature_summary)

    # ------------------------------------------------------------------    
    # remove selected columns and rows, ...

    # Removing outlier features, except the one that provides a birth year
    df.drop(columns=['ALTER_HH', 'KBA05_BAUMAX', 'KK_KUNDENTYP',
        'AGER_TYP', 'TITEL_KZ'], inplace = True)

    # If I'm going to make sense out of these, I need more helpful names

    # Read in the names mapping CSV as a dict
    new_names = pd.read_csv('col_renaming.csv', header=None, index_col=0,
                            squeeze=True).to_dict()

    df.rename(columns=new_names, inplace=True)

    # Remove rows having more than one missing value
    df = df.loc[df.isnull().sum(axis=1) < 2,:]

    
    # -------------------------------------------------------------------
    # select, re-encode, and engineer column values.

    # Re-encode categorical variable(s) to be kept in the analysis.
    cat_cols = ['Bldg: Location Relative to E or W Germany',
               'Small or Home Office Owner', 'Insurance Type',
               'Nationality Based on Name', 'Shopper Type',
                'Socioeconomic Status - LowRes', 'Family Type - LowRes',
                'MoneyType__Primary', 'Energy Consumption Type',
                'Consumption Channel Type', 'Bldg: Building Type',
                'RR4: Life Stage Type - LowRes', 'Socioeconomic Status - HighRes',
                'Family Type - HighRes', 'Vacation Habits', 
                'RR4: Life Stage Type - HighRes']

    # Have to drop first dummy so avoid Dummy Variable Trap
    # Have to include NaN dummy so zero-vector can't be interpreted ambiguously
        # as either NaN or first dropped dummy (which are likely different)
    df = pd.get_dummies(
        df, prefix_sep='__', 
        drop_first=True, dummy_na=True,
        columns=cat_cols)



    def data_mapper(series, mapping_dict):
        '''
        Reads in a pandas Series object that represents the Generation Designation feature
        and returns a re-encoded series according to mapping_dict.

        Inputs
        ------
        series: pandas Series of integer codes (1 through 15) representing different
            Generation Designation values

        mapping_dict: dict of form {designation_code: new_code} used to determine what
            values to return


        Returns
        -------
        pandas Series with the new codes
        '''
        
        # Since NaN values aren't always propagated as expected, do a quick check
        print(f"There are {series.isnull().sum()} null values in the series \
        {series.name} prior to extraction")
        
        out = series.map(mapping_dict, na_action = 'ignore')
        
        print(f"There are {out.isnull().sum()} null values in the series \
        {series.name} after extraction")
        
        return out              


    # For extracting decade of birth info from Generation Designation feature
    decade_code_map = {
    1: 0,
    2: 0,
    3: 1,
    4: 1,
    5: 2,
    6: 2,
    7: 2,
    8: 3,
    9: 3,
    10: 4,
    11: 4,
    12: 4,
    13: 4,
    14: 5,
    15: 5
    }

    df['Generation Decade'] = \
    data_mapper(df['Generation Designation'],
        decade_code_map)

    # For extracting generational movement from Generation Designation feature
    movement_code_map = {
    1: 0,
    2: 1,
    3: 0,
    4: 1,
    5: 0,
    6: 1,
    7: 1,
    8: 0,
    9: 1,
    10: 0,
    11: 1,
    12: 0,
    13: 1,
    14: 0,
    15: 1
    }

    df['Generation Movement'] = \
    data_mapper(df['Generation Designation'],
        movement_code_map)

    # Drop Generation Designation feature now that we've captured
    # its information in the new features
    df.drop(columns='Generation Designation', inplace=True)


    # Cast as int since it seems to keep coming up as object type
    df["RR4: Life Stage Type - Int'l Code Mapping"] =\
    df["RR4: Life Stage Type - Int'l Code Mapping"].astype(int)

    # Extract the tens digit as an int - wealth code
    df["RR4: Life Stage Type - Int'l - Wealth"] = \
    (df["RR4: Life Stage Type - Int'l Code Mapping"] \
        / 10).astype(int)

    # Extract the ones digit - life stage code
    df["RR4: Life Stage Type - Int'l - Stage"] = \
    df["RR4: Life Stage Type - Int'l Code Mapping"] % 10

    # Drop lngering RR4: Life Stage Type - Int'l Code Mapping feature
    df.drop(
        columns="RR4: Life Stage Type - Int'l Code Mapping", inplace=True)

    # Make dummies for Life Stage - HighRes, and Life Stage - LowRes
    cat_cols = ['Life Stage - HighRes',
               'Life Stage - LowRes']

    df = pd.get_dummies(
        df, prefix_sep='__', 
        drop_first=True, dummy_na=True,
        columns=cat_cols)

    # For extracting rural neighborhood status from Neighborhood Quality
    rural_code_map = {
    0: np.nan,
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    7: 1,
    8: 2
    }

    df['Bldg: Rural Type'] = data_mapper(df['Bldg: Neighborhood Quality'],
        rural_code_map)

    # Make dummies out of the multi-level Rural Type categorical
    cat_cols = ['Bldg: Rural Type']

    df = pd.get_dummies(df,
        prefix_sep='__',
        drop_first=True,
        dummy_na=True,
        columns=cat_cols)

    # Exclude rural categories as though they weren't scored
    neighborhood_code_map = {
        0: 0,
        1: 1,
        2: 2,
        3: 3,
        4: 4,
        5: 5,
        7: 0,
        8: 0
        }

    df['Bldg: Neighborhood Quality'] = \
    data_mapper(df['Bldg: Neighborhood Quality'],
        neighborhood_code_map)

    # For extracting business building dominance and dropping info about rest
    biz_code_map = {
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 1
    }
    
    df['PLZ8: Primarily Business Bldgs'] = \
        data_mapper(df['PLZ8: Most Common Bldg Type'],
            biz_code_map)

    # Drop original feature, it's no longer useful
    df.drop(columns=['PLZ8: Most Common Bldg Type'],
                             inplace=True)


    # -------------------------------------------------------------------
    # Return the cleaned dataframe.
    
    return df

In [ ]:
# Test out our new cleaning function

# Load in the general demographics data.
gen_pop = pd.read_csv('data/Udacity_AZDIAS_Subset.csv', sep=';')

# Load in the feature summary file.
feat_info = pd.read_csv('data/AZDIAS_Feature_Summary.csv', sep=';')

gen_pop = clean_data(gen_pop, feat_info)
gen_pop.info()

## Step 2: Feature Transformation

### Step 2.1: Apply Feature Scaling

Before we apply dimensionality reduction techniques to the data, we need to perform feature scaling so that the principal component vectors are not influenced by the natural differences in scale for features. Starting from this part of the project, you'll want to keep an eye on the [API reference page for sklearn](http://scikit-learn.org/stable/modules/classes.html) to help you navigate to all of the classes and functions that you'll need. In this substep, you'll need to check the following:

- sklearn requires that data not have missing values in order for its estimators to work properly. So, before applying the scaler to your data, make sure that you've cleaned the DataFrame of the remaining missing values. This can be as simple as just removing all data points with missing data, or applying an [Imputer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html) to replace all missing values. You might also try a more complicated procedure where you temporarily remove missing values in order to compute the scaling parameters before re-introducing those missing values and applying imputation. Think about how much missing data you have and what possible effects each approach might have on your analysis, and justify your decision in the discussion section below.
- For the actual scaling function, a [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) instance is suggested, scaling each feature to mean 0 and standard deviation 1.
- For these classes, you can make use of the `.fit_transform()` method to both fit a procedure to the data as well as apply the transformation to the data at the same time. Don't forget to keep the fit sklearn objects handy, since you'll be applying them to the customer demographics data towards the end of the project.

**Recalling what we saw towards the end of the preprocessing work, we know that about 60% of our rows have no missing values, with the remaining 40% having at least one missing value.** Additionally, here's a reminder on our current situation with regards to missing values at a feature level:

In [ ]:
missing = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})
missing['percent missing'] = round(missing['total missing'] / len(gen_pop),2)
missing.sort_values('total missing', ascending = False, inplace = True)
missing

**Obviously, the feature that would require the most imputation is `Birth Year`.** While I'm not thrilled about the fact that I'd be imputing this one substantially, my earlier logic on why it needs to be done (it's potentially a very useful age proxy variable, as `Age Bin` is very low resolution with 15-year bins) still holds. 

* All of this being said, if I find questionable results in the clustering produced later, I may opt to drop the `Birth Year` feature after all.

Let's take a look at the distributions of non-missing values for each of these features to get an idea of what we're working with here.

In [ ]:
sns.distplot(gen_pop['Birth Year'].dropna())

In [ ]:
gen_pop['Birth Year'].describe()

In [ ]:
code_map = {
    1: '< 30 years old',
    2: '30 - 45 years old',
    3: '46 - 60 years old',
    4: '> 60 years old'
}

sns.distplot(gen_pop['Age Bin'].dropna())

It looks like the majority `Age Bin` value (45-60 years old) corresponds roughly to the median `Birth Year` value of 1967 (given that the final year in the `Birth Year` feature is 2017, the midpoint of this bin would be 1965). Additionally, the `Birth Year` feature is left-skewed, which corresponds reasonably well with the `Age Bin` having its second peak for the oldest group (code 4, 60+ year olds). **It seems like using `Age Bin` as a way to impute `Birth Year` is a pretty reasonable plan!**

In [ ]:
code_map = {
    1: 'most likely',
    2: 'very likely',
    3: 'likely',
    4: 'average',
    5: 'unlikely',
    6: 'very unlikely'
}

sns.distplot(gen_pop['HH: Probability of Children in Residence'].dropna())

As I can see this feature being correlated with at least a few other features (e.g. `Family Types` and other regional-level family-oriented features), I hesitate to use a simple imputation method like the mode. Also, this feature is missing *relatively* few values (about 0.8% of the dataset). **Thus, I am going to simply drop the rows in which it is missing.**

In [ ]:
sns.distplot(gen_pop['Bldg: Number of HHs'].dropna())

In [ ]:
gen_pop['Bldg: Number of HHs'].describe()

This one feels like it's on the cusp for me, decision-wise. I see a few regional-level features that might correlate with this one, so I'm hesitant to do a quick univariate imputation, but at the same time the amount of missing value is so low that I'm not too worried about improperly skewing the sample relative to the underlying population distribution. **So for this one I think I'll err on the side of not losing more data than I have to and I'll impute using the median** (the mean would be inappropriate, given that the distribution is so skewed).

In [ ]:
sns.distplot(gen_pop['PLZ8: Number of Cars'].dropna())

In [ ]:
gen_pop['PLZ8: Number of Cars'].describe()

Much like the previous feature, I feel like this one has so few missing values that a simple imputation approach wouldn't impact the overall distribution too strongly. And, as a bonus, this one doesn't seem to have any intuitively related features elsewhere, strengthening that argument. **So we'll plan to also impute this one using the median, given its amount of skew.**

In [ ]:
code_map = {
    1: 'building is located in a 125 x 125m grid cell (RA1), which is a consumption cell',
    2: 'building is located in a 250 x 250m grid cell that includes at least one RA1-consumption cell',
    3: 'building is located in a 500 x 500m grid cell that includes at least one RA1-consumption cell',
    4: 'building is located in a 1 x 1km grid cell that includes at least one RA1-consumption cell',
    5: 'building is located in a 2 x 2km grid cell that includes at least one RA1-consumption cell',
    6: 'building is located in a 10 x 10km grid cell that includes at least one RA1-consumption cell',
    7: 'building is not located in a 10 x 10km range of a consumption cell'
}

sns.distplot(gen_pop['Bldg: Distance to Point of Sale Category'].dropna())

This seems to effectively be a distance metric that can be used as a proxy to how urban or business-adjacent a building is. I see no reason to treat this one any differently than those that came before it. **Let's go ahead and just use the mode value of 1 for this one, since it's ordinal (and thus mean or median would be inappropriate).** I'm using the median instead of the mode here since this is ordinal in nature and could roughly be translated into a continuous distance variable with some interpolation.

<center> <b> My Imputation Plan </b> </center>

Feature Name | % of Rows Missing Values | Imputation Approach
--- | --- | ---
Birth Year | 37% | Since Age Bin isn't missing any values, I'll use the midpoint of the bin for a given row as the Birth Year
HH: Probability of Children in Residence | 1% | Drop the rows wherein this is missing
Bldg: Number of HHs	| < 1% | Impute using the median
PLZ8: Number of Cars | < 1% | Impute using the median
Bldg: Distance to Point of Sale Category | < 1% | Impute using mode

In [ ]:
# If you've not yet cleaned the dataset of all NaN values, then investigate and
# do that now.

# Use Age Bin to impute Birth Year values, using midpoint of associated Age Bin to dictate year
age_bin_map = {
    1: '< 30 years old',
    2: '30 - 45 years old',
    3: '46 - 60 years old',
    4: '> 60 years old'
}

# Assume data are all relative to 2017, as this is the maximum Birth Year value
# Earliest Birth Year is 1900, making greatest age 117 
    # Thus we'll assume 20 year midpoint for code 4 (assuming anyone 100+ is outlier)
age_bin_to_year = {
    1: 2002,
    2: 1979,
    3: 1963,
    4: 1937
}

# Make a series that is a mapping of Age Bin to birth years
mapped_series = gen_pop['Age Bin'].map(age_bin_to_year)

# Use this mapped series and fillna() on Birth Year 
    # to take midpoint Age Bin year as Birth Year
gen_pop['Birth Year'].fillna(mapped_series, inplace=True)


In [ ]:
# Recompute missing values

missing = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})
missing['percent missing'] = round(missing['total missing'] / len(gen_pop),2)
missing.sort_values('total missing', ascending = False, inplace = True)
missing

In [ ]:
# Drop null rows for HH: Probability of Children in Residence	
gen_pop.dropna(subset=['HH: Probability of Children in Residence'],
              inplace=True)

In [ ]:
# Recompute missing values

missing = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})
missing['percent missing'] = round(missing['total missing'] / len(gen_pop),2)
missing.sort_values('total missing', ascending = False, inplace = True)
missing

In [ ]:
# Bldg: Number of HHs and PLZ8: Number of Cars - Impute using the median
from sklearn.impute import SimpleImputer

imp_median = SimpleImputer(strategy='median')

gen_pop[['Bldg: Number of HHs', 'PLZ8: Number of Cars']] = \
imp_median.fit_transform(gen_pop[['Bldg: Number of HHs', 
                                  'PLZ8: Number of Cars']])

In [ ]:
# Recompute missing values

missing = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})
missing['percent missing'] = round(missing['total missing'] / len(gen_pop),2)
missing.sort_values('total missing', ascending = False, inplace = True)
missing

In [ ]:
# Bldg: Distance to Point of Sale Category - Impute using mode
imp_mode = SimpleImputer(strategy='most_frequent')

gen_pop['Bldg: Distance to Point of Sale Category'] = \
imp_mode.fit_transform(gen_pop['Bldg: Distance to Point of Sale Category'].values.reshape(-1, 1))

In [ ]:
# Recompute missing values

missing = pd.DataFrame(gen_pop.isnull().sum()).rename(columns = {0: 'total missing'})
missing['percent missing'] = round(missing['total missing'] / len(gen_pop),2)
missing.sort_values('total missing', ascending = False, inplace = True)
missing

**There you go! No more missing values!**

In [ ]:
# Apply feature scaling to the general population demographics data.

# A StandardScaler instance is suggested, scaling each feature to mean 0 and standard deviation 1.



### Discussion 2.1: Apply Feature Scaling

(Double-click this cell and replace this text with your own text, reporting your decisions regarding feature scaling.)

### Step 2.2: Perform Dimensionality Reduction

On your scaled data, you are now ready to apply dimensionality reduction techniques.

- Use sklearn's [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class to apply principal component analysis on the data, thus finding the vectors of maximal variance in the data. To start, you should not set any parameters (so all components are computed) or set a number of components that is at least half the number of features (so there's enough features to see the general trend in variability).
- Check out the ratio of variance explained by each principal component as well as the cumulative variance explained. Try plotting the cumulative or sequential values using matplotlib's [`plot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) function. Based on what you find, select a value for the number of transformed features you'll retain for the clustering part of the project.
- Once you've made a choice for the number of components to keep, make sure you re-fit a PCA instance to perform the decided-on transformation.

In [ ]:
# Apply PCA to the data.



In [ ]:
# Investigate the variance accounted for by each principal component.



In [ ]:
# Re-apply PCA to the data while selecting for number of components to retain.



### Discussion 2.2: Perform Dimensionality Reduction

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding dimensionality reduction. How many principal components / transformed features are you retaining for the next step of the analysis?)

### Step 2.3: Interpret Principal Components

Now that we have our transformed principal components, it's a nice idea to check out the weight of each variable on the first few components to see if they can be interpreted in some fashion.

As a reminder, each principal component is a unit vector that points in the direction of highest variance (after accounting for the variance captured by earlier principal components). The further a weight is from zero, the more the principal component is in the direction of the corresponding feature. If two features have large weights of the same sign (both positive or both negative), then increases in one tend expect to be associated with increases in the other. To contrast, features with different signs can be expected to show a negative correlation: increases in one variable should result in a decrease in the other.

- To investigate the features, you should map each weight to their corresponding feature name, then sort the features according to weight. The most interesting features for each principal component, then, will be those at the beginning and end of the sorted list. Use the data dictionary document to help you understand these most prominent features, their relationships, and what a positive or negative value on the principal component might indicate.
- You should investigate and interpret feature associations from the first three principal components in this substep. To help facilitate this, you should write a function that you can call at any time to print the sorted list of feature weights, for the *i*-th principal component. This might come in handy in the next step of the project, when you interpret the tendencies of the discovered clusters.

In [ ]:
# Map weights for the first principal component to corresponding feature names
# and then print the linked values, sorted by weight.
# HINT: Try defining a function here or in a new cell that you can reuse in the
# other cells.



In [ ]:
# Map weights for the second principal component to corresponding feature names
# and then print the linked values, sorted by weight.



In [ ]:
# Map weights for the third principal component to corresponding feature names
# and then print the linked values, sorted by weight.



### Discussion 2.3: Interpret Principal Components

(Double-click this cell and replace this text with your own text, reporting your observations from detailed investigation of the first few principal components generated. Can we interpret positive and negative values from them in a meaningful way?)

## Step 3: Clustering

### Step 3.1: Apply Clustering to General Population

You've assessed and cleaned the demographics data, then scaled and transformed them. Now, it's time to see how the data clusters in the principal components space. In this substep, you will apply k-means clustering to the dataset and use the average within-cluster distances from each point to their assigned cluster's centroid to decide on a number of clusters to keep.

- Use sklearn's [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) class to perform k-means clustering on the PCA-transformed data.
- Then, compute the average difference from each point to its assigned cluster's center. **Hint**: The KMeans object's `.score()` method might be useful here, but note that in sklearn, scores tend to be defined so that larger is better. Try applying it to a small, toy dataset, or use an internet search to help your understanding.
- Perform the above two steps for a number of different cluster counts. You can then see how the average distance decreases with an increasing number of clusters. However, each additional cluster provides a smaller net benefit. Use this fact to select a final number of clusters in which to group the data. **Warning**: because of the large size of the dataset, it can take a long time for the algorithm to resolve. The more clusters to fit, the longer the algorithm will take. You should test for cluster counts through at least 10 clusters to get the full picture, but you shouldn't need to test for a number of clusters above about 30.
- Once you've selected a final number of clusters to use, re-fit a KMeans instance to perform the clustering operation. Make sure that you also obtain the cluster assignments for the general demographics data, since you'll be using them in the final Step 3.3.

In [ ]:
# Over a number of different cluster counts...


    # run k-means clustering on the data and...
    
    
    # compute the average within-cluster distances.
    
    

In [ ]:
# Investigate the change in within-cluster distance across number of clusters.
# HINT: Use matplotlib's plot function to visualize this relationship.



In [ ]:
# Re-fit the k-means model with the selected number of clusters and obtain
# cluster predictions for the general population demographics data.



### Discussion 3.1: Apply Clustering to General Population

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding clustering. Into how many clusters have you decided to segment the population?)

### Step 3.2: Apply All Steps to the Customer Data

Now that you have clusters and cluster centers for the general population, it's time to see how the customer data maps on to those clusters. Take care to not confuse this for re-fitting all of the models to the customer data. Instead, you're going to use the fits from the general population to clean, transform, and cluster the customer data. In the last step of the project, you will interpret how the general population fits apply to the customer data.

- Don't forget when loading in the customers data, that it is semicolon (`;`) delimited.
- Apply the same feature wrangling, selection, and engineering steps to the customer demographics using the `clean_data()` function you created earlier. (You can assume that the customer demographics data has similar meaning behind missing data patterns as the general demographics data.)
- Use the sklearn objects from the general demographics data, and apply their transformations to the customers data. That is, you should not be using a `.fit()` or `.fit_transform()` method to re-fit the old objects, nor should you be creating new sklearn objects! Carry the data through the feature scaling, PCA, and clustering steps, obtaining cluster assignments for all of the data in the customer demographics data.

In [ ]:
# Load in the customer demographics data.
customers = 

In [ ]:
# Apply preprocessing, feature transformation, and clustering from the general
# demographics onto the customer data, obtaining cluster predictions for the
# customer demographics data.



### Step 3.3: Compare Customer Data to Demographics Data

At this point, you have clustered data based on demographics of the general population of Germany, and seen how the customer data for a mail-order sales company maps onto those demographic clusters. In this final substep, you will compare the two cluster distributions to see where the strongest customer base for the company is.

Consider the proportion of persons in each cluster for the general population, and the proportions for the customers. If we think the company's customer base to be universal, then the cluster assignment proportions should be fairly similar between the two. If there are only particular segments of the population that are interested in the company's products, then we should see a mismatch from one to the other. If there is a higher proportion of persons in a cluster for the customer data compared to the general population (e.g. 5% of persons are assigned to a cluster for the general population, but 15% of the customer data is closest to that cluster's centroid) then that suggests the people in that cluster to be a target audience for the company. On the other hand, the proportion of the data in a cluster being larger in the general population than the customer data (e.g. only 2% of customers closest to a population centroid that captures 6% of the data) suggests that group of persons to be outside of the target demographics.

Take a look at the following points in this step:

- Compute the proportion of data points in each cluster for the general population and the customer data. Visualizations will be useful here: both for the individual dataset proportions, but also to visualize the ratios in cluster representation between groups. Seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) or [`barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) function could be handy.
  - Recall the analysis you performed in step 1.1.3 of the project, where you separated out certain data points from the dataset if they had more than a specified threshold of missing values. If you found that this group was qualitatively different from the main bulk of the data, you should treat this as an additional data cluster in this analysis. Make sure that you account for the number of data points in this subset, for both the general population and customer datasets, when making your computations!
- Which cluster or clusters are overrepresented in the customer dataset compared to the general population? Select at least one such cluster and infer what kind of people might be represented by that cluster. Use the principal component interpretations from step 2.3 or look at additional components to help you make this inference. Alternatively, you can use the `.inverse_transform()` method of the PCA and StandardScaler objects to transform centroids back to the original data space and interpret the retrieved values directly.
- Perform a similar investigation for the underrepresented clusters. Which cluster or clusters are underrepresented in the customer dataset compared to the general population, and what kinds of people are typified by these clusters?

In [ ]:
# Compare the proportion of data in each cluster for the customer data to the
# proportion of data in each cluster for the general population.



In [ ]:
# What kinds of people are part of a cluster that is overrepresented in the
# customer data compared to the general population?



In [ ]:
# What kinds of people are part of a cluster that is underrepresented in the
# customer data compared to the general population?



### Discussion 3.3: Compare Customer Data to Demographics Data

(Double-click this cell and replace this text with your own text, reporting findings and conclusions from the clustering analysis. Can we describe segments of the population that are relatively popular with the mail-order company, or relatively unpopular with the company?)

> Congratulations on making it this far in the project! Before you finish, make sure to check through the entire notebook from top to bottom to make sure that your analysis follows a logical flow and all of your findings are documented in **Discussion** cells. Once you've checked over all of your work, you should export the notebook as an HTML document to submit for evaluation. You can do this from the menu, navigating to **File -> Download as -> HTML (.html)**. You will submit both that document and this notebook for your project submission.